In [ ]:
import numpy as np
import matplotlib.pyplot as plt





In [ ]:
import pandas as pd

# Game setup
num_cooperators = 4
num_defectors = 5-num_cooperators
num_events = 5

# Expected cooperative event score by number of cooperators
E_event_coop = {
    4: 3.5,
    3: 3,
    2: 2,
    1: 1,
    0: 0
}

# Expected defector contribution per event
E_event_risky_d = {
    1: 3 / num_events,
    2: 6 / num_events,
    3: 9 / num_events,
    4: 12 / num_events
}

E_event_safe_d = {
    1: 2 / num_events,
    2: 5 / num_events,
    3: 8 / num_events,
    4: 11 / num_events
}

# Score modifiers for outcomes
coop_score = {
    "deflected": -5,
    "unknown": 0,
    "found": 5
}

defector_score = {
    "deflected": 6,
    "unknown": 1,
    "found": -10
}

# Event 5 bonuses
mission_complete_bonus = 10         # Coops get this unless sabotaged
mission_sabotaged_bonus = 10        # Risky unknown defector gets this

# Scenarios: (defector_type, outcome)
scenarios = [
    ("safe", "deflected"),
    ("safe", "unknown"),
    ("safe", "found"),
    ("risky", "deflected"),
    ("risky", "unknown"),
    ("risky", "found")
]

scenario_names = [f"{typ}-{outcome}" for typ, outcome in scenarios]
event_names = [f"Event {i}" for i in range(1, num_events + 1)]

# Initialize tables
coop_table = pd.DataFrame(index=scenario_names, columns=event_names)
def_table = pd.DataFrame(index=scenario_names, columns=event_names)

# Populate tables
for i, (typ, outcome) in enumerate(scenarios):
    for event_ind in range(1, num_events + 1):
        # Coop score: base + outcome effect
        coop_score_event = event_ind*E_event_coop[num_cooperators] 
        coop_score_event /= num_cooperators
        coop_score_event += coop_score[outcome]
        if event_ind == 5:
            coop_score_event += mission_complete_bonus

        # Defector score: base + outcome effect
        def_event_base = (E_event_risky_d if typ == "risky" else E_event_safe_d).get(event_ind, 0)
        def_score_event = event_ind*def_event_base 
        def_score_event /= num_defectors
        def_score_event += defector_score[outcome]
        if event_ind == 5 and outcome != "found" and typ == "risky":
            def_score_event += mission_sabotaged_bonus

        coop_table.iloc[i, event_ind - 1] = coop_score_event
        def_table.iloc[i, event_ind - 1] = def_score_event

# Output results
print("Expected score diffs:\n", coop_table - def_table)
#print("Expected Cooperator Scores:\n", coop_table)
#print("\nExpected Defector Scores:\n", def_table)





In [ ]:
import pandas as pd
import numpy as np

def make_tables(n_coop, coop_mod, def_mod, bonus_coop, bonus_def, def_starting_score, n_events=5):
    n_def = 5 - n_coop

    # base scores
    E_event_coop = {
        4:3.5,
        3:3,
        2:2,
        1:1}
    E_event_coop = {k: v/k for k, v in E_event_coop.items()}
    E_event_coop[0] = 0
    E_risky = {1:(3/n_events) + def_starting_score,
               2:(6/n_events) + def_starting_score,
               3:(9/n_events) + def_starting_score,
               4:(12/n_events) + def_starting_score,}
    E_risky = {k : v/k for k, v in E_risky.items()}
    
    E_safe  = {1:(2/n_events) + def_starting_score,
               2:(5/n_events) + def_starting_score,
               3:(8/n_events) + def_starting_score,
               4:(11/n_events) + def_starting_score}
    E_safe  = {k : v/k for k, v in E_safe.items()}
    



    scenarios = [("safe","deflected"),("safe","unknown"),("safe","found"),
                 ("risky","deflected"),("risky","unknown"),("risky","found")]

    names = [f"{t}-{o}" for t,o in scenarios]
    evs   = [f"Event {i}" for i in range(1, n_events+1)]
    coop_tbl = pd.DataFrame(index=names, columns=evs, dtype=float)
    def_tbl  = coop_tbl.copy()

    for idx,(typ,outcome) in enumerate(scenarios):
        for e in range(1, n_events+1):
            # cooperators
            base_c = E_event_coop.get(n_coop, 0)
            c = (e*base_c) + coop_mod[outcome]
            if e==5 :
                c += bonus_coop
            coop_tbl.iat[idx, e-1] = c

            # defectors
            base_d = (E_risky if typ=="risky" else E_safe).get(n_def, 0)
            d = (e*base_d) + def_mod[outcome]
            if e==5 and typ=="risky" and outcome!="found":
                d += bonus_def
            def_tbl.iat[idx, e-1] = d

    return coop_tbl, def_tbl

def check_constraints(coop_tbl, def_tbl):
    fails = []

    # shorthand
    c = coop_tbl
    d = def_tbl
    ev5 = "Event 5"

    # defector ordering constraints
    for kind in ("risky","safe"):
        a, b, f = f"{kind}-deflected", f"{kind}-unknown", f"{kind}-found"
        if not (d.at[a,ev5] > d.at[b,ev5] > d.at[f,ev5]):
            fails.append(f"Defector {kind}: {a} > {b} > {f} at {ev5}")

    # risky vs safe for each outcome
    for outcome in ("deflected","unknown","found"):
        r,s = f"risky-{outcome}", f"safe-{outcome}"
        if not (d.at[r,ev5] >= d.at[s,ev5]):
            fails.append(f"Defector risky vs safe for {outcome} at {ev5}")

    # cooperator ordering constraints
    for kind in ("risky","safe"):
        a, b, f = f"{kind}-deflected", f"{kind}-unknown", f"{kind}-found"
        if not (c.at[f,ev5] > c.at[b,ev5] > c.at[a,ev5]):
            fails.append(f"Cooperator {kind}: {f} > {b} > {a} at {ev5}")
            
    # deflected and unknown should always be negative for events 1,2,3,4
    for e in range(1, 5):
        for kind in ("risky","safe"):
            for outcome in ("deflected","unknown"):
                d_val = d.at[f"{kind}-{outcome}", f"Event {e}"]
                c_val = c.at[f"{kind}-{outcome}", f"Event {e}"]
                if c_val > d_val:
                    fails.append(f"Defector {kind}: {outcome} not negative at Event {e}")
           
    return fails


coop_mod = {"deflected":-5, "unknown":0, "found":5}
def_mod  = {"deflected":5,  "unknown":0, "found":-5}

coop_event_finished = 5
def_event_sabotage  = 10

def_starting_score = 1

#loop over floor, round and ceil operators
#for op in ("none", "floor", "round", "ceil"):
for op in ("ceil", ):
    # main loop over different cooperator counts
    for n_coop in [4,3,2,1]:
        coop_tbl, def_tbl = make_tables(n_coop, coop_mod, def_mod, coop_event_finished, def_event_sabotage, def_starting_score)
        if op == "floor":
            coop_tbl = coop_tbl.apply(np.floor)
            def_tbl  = def_tbl.apply(np.floor)
        elif op == "round":
            coop_tbl = coop_tbl.apply(np.round)
            def_tbl  = def_tbl.apply(np.round)
        elif op == "ceil":
            coop_tbl = coop_tbl.apply(np.ceil)
            def_tbl  = def_tbl.apply(np.ceil)
        
        diff_tbl = coop_tbl - def_tbl

        print(f"\n=== n_cooperators = {n_coop} ===")
        #print("Cooperator table:\n", coop_tbl)
        #print("\nDefector table:\n", def_tbl)
        
        print("\nDiff table:\n", diff_tbl)

        errors = check_constraints(coop_tbl, def_tbl)
        if errors:
            print(f"\nConstraint failures ({op}):")
            for e in errors:
                print("  -", e)
        else:
            print("\nAll constraints passed.")



=== n_cooperators = 4 ===

Diff table:
                  Event 1  Event 2  Event 3  Event 4  Event 5
safe-deflected     -11.0    -11.0    -12.0    -12.0     -7.0
safe-unknown        -1.0     -1.0     -2.0     -2.0      3.0
safe-found           9.0      9.0      8.0      8.0     13.0
risky-deflected    -11.0    -12.0    -12.0    -13.0    -18.0
risky-unknown       -1.0     -2.0     -2.0     -3.0     -8.0
risky-found          9.0      8.0      8.0      7.0     12.0

All constraints passed.

=== n_cooperators = 3 ===

Diff table:
                  Event 1  Event 2  Event 3  Event 4  Event 5
safe-deflected     -10.0    -10.0    -10.0    -10.0     -5.0
safe-unknown         0.0      0.0      0.0      0.0      5.0
safe-found          10.0     10.0     10.0     10.0     15.0
risky-deflected    -11.0    -11.0    -11.0    -11.0    -16.0
risky-unknown       -1.0     -1.0     -1.0     -1.0     -6.0
risky-found          9.0      9.0      9.0      9.0     14.0

All constraints passed.

=== n_coopera